In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import glob
from PIL import Image
import numpy as np
import os
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tqdm import tqdm

from keras.models import Sequential, Model, load_model
from keras.layers import UpSampling2D, Conv2D, Activation, BatchNormalization, Reshape, Dense, Input, LeakyReLU, Dropout, Flatten, ZeroPadding2D
from keras.optimizers import Adam,SGD

Using TensorFlow backend.


In [3]:
def build_generator():
    noise_shape = (100,)
    model = Sequential()
    model.add(Dense(64 * (192 // (2 ** 5)) * (256 // (2 ** 5)), activation="relu", input_shape=noise_shape))
    model.add(Reshape(((192 // (2 ** 5)), (256 // (2 ** 5)), 64)))
    model.add(BatchNormalization(momentum=0.8))

    model.add(UpSampling2D())
    model.add(Conv2D(1024, (3, 3), padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(UpSampling2D())
    model.add(Conv2D(512, (3, 3), padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(UpSampling2D())
    model.add(Conv2D(256, (3, 3), padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(UpSampling2D())
    model.add(Conv2D(128, (3, 3), padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(UpSampling2D())
    model.add(Conv2D(64, (3, 3), padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2D(32, (3, 3), padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2D(3, (3, 3), padding = 'same'))
    model.add(Activation('tanh'))

    noise = Input(shape = noise_shape)
    img = model(noise)

    return model

def build_discriminator():
        
    img_shape = (192, 256, 3)

    model = Sequential()

    model.add(Conv2D(32, (3, 3), strides = 2, input_shape=img_shape, padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), strides = 2, padding = 'same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2D(128, (3, 3), strides = 2, padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2D(256, (3, 3), strides = 1, padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
        
    model.add(Conv2D(512, (3, 3), strides = 1, padding = 'same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation = 'sigmoid'))

    return model

def combine_model(g,d):
    model = Sequential()
    model.add(g)
    d.trainable = False
    model.add(d)
    return model

def get_data(imgPath):
    basePath = '/content/drive/My Drive/Colab Notebooks/DCGAN-data/'
    X = []
    for path in tqdm(imgPath):
        img = Image.open(basePath + path)
        img = np.asarray(img)
        X.append(img)
    return np.asarray(X)

def gene_imgs(count,g):
    noise = np.random.normal(0, 1, (count, 100))
    return g.predict(noise)
    
def save_imgs(epoch,g):
    r, c = 5, 5

    imgs = gene_imgs(r*c,g)
    imgs = 127.5 * imgs + 127.5

    nindex, height, width, intensity = imgs.shape
    nrows = nindex // c
    assert nindex == nrows * c
        
    gallery = (imgs.reshape(nrows, c, height, width, intensity)
                  .swapaxes(1, 2)
                  .reshape(height * nrows, width * c, intensity))
    path = '/content/drive/My Drive/Colab Notebooks/gantest' + '/gallery'
    if not os.path.exists(path):
        os.makedirs(path)
    cv2.imwrite(path + f"/{epoch}.jpg", gallery)

def trainModel():
    pathList = os.listdir('/content/drive/My Drive/Colab Notebooks/DCGAN-data/')
    X_train = get_data(pathList)

    X_train = (X_train.astype(np.float32) - 127.5) / 127.5

    BATCH_SIZE = 16
    
    d = build_discriminator()
    g = build_generator()
    d_on_g = combine_model(g, d)

    d_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    g.compile(loss='binary_crossentropy', optimizer="SGD")
    d_on_g.compile(loss='binary_crossentropy', optimizer=g_optim)
    d.trainable = True
    d.compile(loss='binary_crossentropy', optimizer=d_optim)
    losses = np.zeros((47, 2))
    for epoch in tqdm(range(10000)):
        print("Epoch is", epoch)
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = g.predict(noise, verbose=0)
            X = np.concatenate((image_batch, generated_images))
            y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
            d_loss = d.train_on_batch(X, y)
            losses[index-1, 0] = d_loss
            noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
            d.trainable = False
            g_loss = d_on_g.train_on_batch(noise, [1] * BATCH_SIZE)
            d.trainable = True
            losses[index-1, 1] = g_loss
        if epoch % 20 == 0:
            print("Epoch is", epoch)
            print('d_loss',np.mean(losses[:, 0]),'g_loss',np.mean(losses[:, 1]))
            save_imgs(epoch,g)
            g.save_weights('/content/drive/My Drive/Colab Notebooks/model/generator.h5', True)
            d.save_weights('/content/drive/My Drive/Colab Notebooks/model/discriminator.h5', True)

In [ ]:
trainModel()

 19%|█▉        | 143/756 [00:47<03:27,  2.95it/s]